# Slipstream GLM-Z1-9B Finetuning & GGUF Export

This notebook demonstrates how to finetune `zai-org/GLM-Z1-9B-0414` on the Slipstream dataset and export it to GGUF format.

In [ ]:
%%capture
# 1. Install Unsloth and Dependencies
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps trl peft accelerate bitsandbytes xformers

In [ ]:
# 2. Setup & Load Base Model
import torch
from unsloth import FastLanguageModel

max_seq_length = 2048
dtype = None # Auto-detect
load_in_4bit = True # 4-bit quantization

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "zai-org/GLM-Z1-9B-0414",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

# Add LoRA adapters
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
)

In [ ]:
# 3. Load & Format Dataset
from datasets import load_dataset
from unsloth.chat_templates import get_chat_template, standardize_sharegpt

# Load from the Hugging Face hub
dataset = load_dataset("anthonym21/slipstream-think-quantize-transmit", split="train")

# Standardize format
dataset = standardize_sharegpt(dataset)
tokenizer = get_chat_template(tokenizer, chat_template = "chatml")

def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts }

dataset = dataset.map(formatting_prompts_func, batched = True)

In [ ]:
# 4. Train
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 1,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

trainer_stats = trainer.train()

In [ ]:
# 5. Inference Test
FastLanguageModel.for_inference(model)

messages = [
    {"role": "system", "content": "You are an AI agent that communicates using the Slipstream protocol (SLIP)."},
    {"role": "user", "content": "Tell bob to review my code"}
]
inputs = tokenizer.apply_chat_template(messages, tokenize = True, add_generation_prompt = True, return_tensors = "pt").to("cuda")

outputs = model.generate(input_ids = inputs, max_new_tokens = 128, use_cache = True)
print(tokenizer.batch_decode(outputs))

In [ ]:
# 6. Save & Export
# Save LoRA adapters
model.save_pretrained("outputs/lora_model")
tokenizer.save_pretrained("outputs/lora_model")

# Save Merged 16-bit Model
model.save_pretrained_merged("outputs/merged_model", tokenizer, save_method = "merged_16bit")

# Save GGUF (Q8_0)
model.save_pretrained_gguf("outputs/gguf_model", tokenizer, quantization_method = "q8_0")